In [1]:
#gs://usp_pos_doc_sid/mupe_coling/mupe_test_metadata01.csv
#gs://usp_pos_doc_sid/mupe/models/mupe-v2-distil-tun-test-output-cer-wer.tsv
#!gsutil cp gs://usp_pos_doc_sid/mupe/models/mupe-v2-distil-tun-test-output-cer-wer.tsv .
!gsutil cp gs://usp_pos_doc_sid/mupe_coling/mupe-v2-distil-tun-test-output-cer-wer-31k.tsv .
!gsutil cp gs://usp_pos_doc_sid/mupe_coling/mupe_test_metadata01.csv .

Copying gs://usp_pos_doc_sid/mupe_coling/mupe-v2-distil-tun-test-output-cer-wer-31k.tsv...
/ [1 files][  9.5 MiB/  9.5 MiB]                                                
Operation completed over 1 objects/9.5 MiB.                                      
Copying gs://usp_pos_doc_sid/mupe_coling/mupe_test_metadata01.csv...
/ [1 files][  1.7 KiB/  1.7 KiB]                                                
Operation completed over 1 objects/1.7 KiB.                                      


In [2]:
import pandas as pd
pd.options.mode.copy_on_write = True
dfm = pd.read_csv('mupe_test_metadata01.csv')
dfm['mupe_code'] = dfm['mupe_code'].str.lower()
dfm.loc[dfm['age']==0,'age'] = 99
dfm.head()

,mupe_code,gender,state,city,age,int_migration,education
0,pc_ma_hv032,M,RS,Rio Grande,55,Y,Superior Incompleto
1,pc_ma_hv212,F,RS,Bagé,22,Y,Superior Incompleto
2,pc_ma_hv210,M,PR,Rolândia,63,Y,Nível Técnico
3,pc_ma_hv260,F,PR,Curitiba,41,Y,Superior Completo
4,pc_ma_hv257,M,GO,Catalão,73,Y,Ensino Fundamental Completo


In [3]:
dfm['age_group'] = pd.cut(dfm['age'], bins=[0, 40, 60, float('inf')],
                         labels=['0-40', '40-60', '60+'],
                         include_lowest=True)

regioes = {
    "RS": "Sul/Sudeste",
    "PR": "Sul/Sudeste",
    "GO": "Norte/Centro-Oeste",
    "MS": "Norte/Centro-Oeste",
    "RO": "Norte/Centro-Oeste",
    "PA": "Norte/Centro-Oeste",
    "PI": "Nordeste",
    "SE": "Nordeste",
    "AL": "Nordeste",
    "PB": "Nordeste",
    "CE": "Nordeste",
    "BA": "Nordeste",
    "ES": "Sul/Sudeste",
    "RJ": "Sul/Sudeste",
    "MG": "Sul/Sudeste",
    "PE": "Nordeste",
    "SP": "Sul/Sudeste",
}
dfm['region'] = dfm['state'].map(regioes)


dfm['education_group'] = dfm['education'].apply(lambda x:
                                            'Ensino Médio' if 'Médio' in x or 'Nível' in x else
                                            'Ensino Superior' if 'Superior' in x or 'Mestrado' in x else
                                            'Ensino Fundamental')

dfm.head()

,mupe_code,gender,state,city,age,int_migration,education,age_group,region,education_group
0,pc_ma_hv032,M,RS,Rio Grande,55,Y,Superior Incompleto,40-60,Sul/Sudeste,Ensino Superior
1,pc_ma_hv212,F,RS,Bagé,22,Y,Superior Incompleto,0-40,Sul/Sudeste,Ensino Superior
2,pc_ma_hv210,M,PR,Rolândia,63,Y,Nível Técnico,60+,Sul/Sudeste,Ensino Médio
3,pc_ma_hv260,F,PR,Curitiba,41,Y,Superior Completo,40-60,Sul/Sudeste,Ensino Superior
4,pc_ma_hv257,M,GO,Catalão,73,Y,Ensino Fundamental Completo,60+,Norte/Centro-Oeste,Ensino Fundamental


In [4]:
print(dfm.groupby('education_group')['mupe_code'].count())
print(dfm.groupby('age_group')['mupe_code'].count())
print(dfm.groupby('region')['mupe_code'].count())
print(dfm.groupby('gender')['mupe_code'].count())

education_group
Ensino Fundamental    15
Ensino Médio           5
Ensino Superior       10
Name: mupe_code, dtype: int64
age_group
0-40      8
40-60     9
60+      13
Name: mupe_code, dtype: int64
region
Nordeste              12
Norte/Centro-Oeste     7
Sul/Sudeste           11
Name: mupe_code, dtype: int64
gender
F    16
M    14
Name: mupe_code, dtype: int64


<ipython-input-4-71bb9638d6ac>:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(dfm.groupby('age_group')['mupe_code'].count())


In [5]:
dfw = pd.read_csv('mupe-v2-distil-tun-test-output-cer-wer-31k.tsv', sep='\t')
dfw.head()

,audio_name,speaker_type,speaker_code,speaker_gender,file_path,original,prediction,original normalized,prediction normalized,wer,cer
0,pc_ma_hv222,R,MA_HV222,M,ma_hv222_1282_4634.372_4643.986.wav,"O dinheiro para eles eu tinha, porque a gente ...","Eu pôr dinheiro para eles, eu tinha, porque a ...",o dinheiro para eles eu tinha porque a gente n...,eu pôr dinheiro para eles eu tinha porque a ge...,0.125000,0.078947
1,pc_ma_hv222,R,MA_HV222,M,ma_hv222_745_2691.687_2695.188.wav,"E eu estava com vinte e dois, e fiquei.","E eu estava com vinte e dois, aí fiquei.",e eu estava com vinte e dois e fiquei,e eu estava com vinte e dois aí fiquei,0.111111,0.054054
2,pc_ma_hv222,R,MA_HV222,M,ma_hv222_441_1637.7120000000002_1642.054.wav,"aquela pessoa que toma conta, falou, deixa eu ...","Aquela pessoa que toma conta, falou, deixa eu ...",aquela pessoa que toma conta falou deixa eu ti...,aquela pessoa que toma conta falou deixa eu ti...,0.000000,0.000000
3,pc_ma_hv222,R,MA_HV222,M,ma_hv222_1099_3971.5860000000002_3977.268.wav,"E a mulher que morreu, deixou... Quantos filhos?",E a mulher que morreu deixou quantos filhos?,e a mulher que morreu deixou quantos filhos,e a mulher que morreu deixou quantos filhos,0.000000,0.000000
4,pc_ma_hv222,R,MA_HV222,M,ma_hv222_1016_3671.625_3673.726.wav,Fui lá e encontrei outro carro.,Foi lá e encontrei outra carro.,fui lá e encontrei outro carro,foi lá e encontrei outra carro,0.333333,0.066667


In [6]:
# prompt: join the two dataframes by key mupe_code equals audio_name
df = dfm.merge(dfw, left_on='mupe_code', right_on='audio_name')
df.head()


,mupe_code,gender,state,city,age,int_migration,education,age_group,region,education_group,...,speaker_type,speaker_code,speaker_gender,file_path,original,prediction,original normalized,prediction normalized,wer,cer
0,pc_ma_hv032,M,RS,Rio Grande,55,Y,Superior Incompleto,40-60,Sul/Sudeste,Ensino Superior,...,R,MA_HV032,M,ma_hv032_431_2203.637_2213.385.wav,"Para o que eu tocava, para o meu conhecimento ...","Para o que eu tocava, para o meu conhecimento ...",para o que eu tocava para o meu conhecimento m...,para o que eu tocava para o meu conhecimento m...,0.000000,0.000000
1,pc_ma_hv032,M,RS,Rio Grande,55,Y,Superior Incompleto,40-60,Sul/Sudeste,Ensino Superior,...,R,MA_HV032,M,ma_hv032_975_4946.229_4950.972.wav,Porque muitos pais também não apoiam tão tanto...,Porque muitos pais também não apoiam tão tanto...,porque muitos pais também não apoiam tão tanto...,porque muitos pais também não apoiam tão tanto...,0.000000,0.000000
2,pc_ma_hv032,M,RS,Rio Grande,55,Y,Superior Incompleto,40-60,Sul/Sudeste,Ensino Superior,...,R,MA_HV032,M,ma_hv032_144_842.787_844.028.wav,Mas eles tocam alguma coi>,Mas eles tocam alguma coisa?,mas eles tocam alguma coi,mas eles tocam alguma coisa,0.200000,0.080000
3,pc_ma_hv032,M,RS,Rio Grande,55,Y,Superior Incompleto,40-60,Sul/Sudeste,Ensino Superior,...,R,MA_HV032,M,ma_hv032_1101_5656.13_5659.592.wav,O cara falar isso para você e ver,O cara falar isso para você e ver,o cara falar isso para você e ver,o cara falar isso para você e ver,0.000000,0.000000
4,pc_ma_hv032,M,RS,Rio Grande,55,Y,Superior Incompleto,40-60,Sul/Sudeste,Ensino Superior,...,R,MA_HV032,M,ma_hv032_153_882.645_887.349.wav,Inclusive tem uma história muito im muito impo...,Inclusive tem uma história muito em muito impo...,inclusive tem uma história muito im muito impo...,inclusive tem uma história muito em muito impo...,0.176471,0.074468


In [7]:
dfr = df[df['speaker_type'] == 'R']

In [8]:
print(dfr.groupby('education_group')['wer'].mean())
print("--------------")
print(dfr.groupby('age_group')['wer'].mean())
print("--------------")
print(dfr.groupby('region')['wer'].mean())
print("--------------")
print(dfr.groupby('gender')['wer'].mean())

education_group
Ensino Fundamental    0.195870
Ensino Médio          0.137157
Ensino Superior       0.114497
Name: wer, dtype: float64
--------------
age_group
0-40     0.142857
40-60    0.118926
60+      0.194707
Name: wer, dtype: float64
--------------
region
Nordeste              0.167129
Norte/Centro-Oeste    0.181421
Sul/Sudeste           0.143528
Name: wer, dtype: float64
--------------
gender
F    0.149764
M    0.171119
Name: wer, dtype: float64


<ipython-input-8-aed25fb52374>:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(dfr.groupby('age_group')['wer'].mean())
